In [65]:
import boto3
from boto3.dynamodb.conditions import Key

import requests
import json
from decimal import Decimal

import argparse


dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('properties_enriched')
s3 = boto3.resource('s3')


## Get list of sold properties

In [52]:
def get_sold(city, state):
    url = "https://realty-in-us.p.rapidapi.com/properties/v2/list-sold"

    querystring = {"offset":"0","limit":"200","city":city,"state_code":state,"sort":"sold_date"}
    headers = {
        'x-rapidapi-host': "realty-in-us.p.rapidapi.com",
        'x-rapidapi-key': "4519f6dcffmshfadff8b94661096p1989c5jsn14919517996b"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()




## Get Item from Table & Save to S3

In [62]:
def save_from_dynamo_to_s3(property_id):

    print(f"INFO: getting from dynamoDB and saving to s3 for property_id: {property_id}")
    response = table.get_item(Key={'property_id': property_id})


    def default(obj):
        if isinstance(obj, Decimal):
            return str(obj)
        raise TypeError("Object of type '%s' is not JSON serializable" % type(obj).__name__)



    json_object = json.dumps(response,default=default) 

    s3object = s3.Object('pb-sold', f'{property_id}.json')

    s3object.put(
        Body=(bytes(json_object.encode('UTF-8')))
    )
    print(f"Done...")




# Delete an item from DynamoDB

In [56]:
def dynamo_delete_property(property_id):
    print(f"INFO: trying to remove property_id {property_id} from dynamodb table")
    response = table.delete_item(
                Key={
                    'property_id': property_id,
                }
            )

    print(response)
    print(f"INFO: removed {property_id} from dynamoddb table")
    return None

In [66]:
def main(city, state):
    print(f"INFO: cleaning up dynamoDb for {city}, {state}")
    sold = get_sold(city=city, state=state)

    for prop in sold['properties']:
        property_id = prop['property_id']
        save_from_dynamo_to_s3(property_id=property_id)
        dynamo_delete_property(property_id=property_id)

    print("Done....")
    
    return None





In [67]:
main(city="Cleveland", state="OH")

INFO: cleaning up dynamoDb for Cleveland, OH
INFO: getting from dynamoDB and saving to s3 for property_id: O4546814815
Done...
INFO: trying to remove property_id O4546814815 from dynamodb table
{'ResponseMetadata': {'RequestId': '0V44JTRPI5V7T52RCRLUEUJDQRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 24 Jan 2022 05:41:13 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '0V44JTRPI5V7T52RCRLUEUJDQRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
INFO: removed O4546814815 from dynamoddb table
INFO: getting from dynamoDB and saving to s3 for property_id: O3693659699
Done...
INFO: trying to remove property_id O3693659699 from dynamodb table
{'ResponseMetadata': {'RequestId': 'PNE3CN2DSR14LGNBTOHKESBIMNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 24 Jan 2022 05:41:13 GMT', 'con